## Bayes算法概述
```
贝叶斯定理是关于随机事件A和B的条件概率定理，即在条件B发生时A发生的概率
```
### bayes公式
<img src="Bayes公式.png" style='zoom:100%' >

### bayes公式物理含义
```
邮件中出现词A，通过此可视现象推断当前邮件为垃圾邮件的概率：
首先加入垃圾邮件和普通邮件出现概率相同时可以计算词A出现的概率：PA+PB。如果两种邮件出现概率不同，则需乘以对应概率即可。
然后计算词A在垃圾邮件中出现的概率PA
最后使用PA/(PA+PB) 即可得到词A出现时，当前邮件为垃圾邮件的概率
```
<img src="bayes公式图形.png" style='zoom:50%' >

### 朴素贝叶斯
```
朴素贝叶斯要求条件相对独立
```

## 1.基于贝叶斯算法的单词纠错实例
```
总体流程：如下图所示。
单词的权重计算：本文以计算单词在预料库中出现的次数为单词权重。单词权重计算还可以与神经网络结合，即通过神经网络学习当前语境信息，然后根据语境信息确定当前词汇权重值。

本文基于预料库计算时，预料库需要根据个人近期阅读和书写内容实时更新预料库内容，如果预料库信息不全时则会导致预测准确率降低。
```
<img src="基于贝叶斯单词纠错流程.png" style='zoom:50%' >

In [9]:
import re,collections

# 语料库预处理
def words(text):
    return re.findall('[a-z]+', text.lower())

# 语料词汇数量统计
def word_record_get(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = word_record_get(words(open('big.txt').read()))       #基于语料库的单词次数查询统计字典

def edits1(word):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    n = len(word)
    return set([word[0:i]+word[i+1:] for i in range(n)] +                     # deletion
               [word[0:i]+word[i+1]+word[i]+word[i+2:] for i in range(n-1)] + # transposition
               [word[0:i]+c+word[i+1:] for i in range(n) for c in alphabet] + # alteration
               [word[0:i]+c+word[i:] for i in range(n+1) for c in alphabet])  # insertion

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word] #可能的单词
    return max(candidates, key=lambda w: NWORDS[w])        #从字典中查看当前单词出现次数

In [11]:
correct('wordd')

'words'

## 2.基于贝叶斯算法的新闻分类

In [ ]:
# 读取数据
import pandas as pd
import jieba
df_news = pd.read_table('./data/val.txt',names=['category','theme','URL','content'],encoding='utf-8')
df_news = df_news.dropna()
df_news.head()

In [ ]:
# 使用结巴分词器分词 将所有文档保存于content_S
content = df_news.content.values.tolist()
content_S = []
for line in content:
    current_segment = jieba.lcut(line)
    if len(current_segment) > 1 and current_segment != '\r\n': #换行符
        content_S.append(current_segment)

#将文档数据转换为DateFrame格式      
df_content=pd.DataFrame({'content_S':content_S})

#读取停用词表 将文档通过停用词表过滤  去掉 词频/逆词频 值较小的词
stopwords=pd.read_csv("stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')

def drop_stopwords(contents,stopwords):
    contents_clean = []
    all_words = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word))
        contents_clean.append(line_clean)
    return contents_clean,all_words

contents = df_content.content_S.values.tolist()    
stopwords = stopwords.stopword.values.tolist()
contents_clean,all_words = drop_stopwords(contents,stopwords)

#将清理后的词转换为DataFrame格式
df_content=pd.DataFrame({'contents_clean':contents_clean})

#统计文档中所有出现的词 创建词云
df_all_words=pd.DataFrame({'all_words':all_words})
words_count=df_all_words.groupby(by=['all_words'])['all_words'].agg({"count":numpy.size})
words_count=words_count.reset_index().sort_values(by=["count"],ascending=False)
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)

wordcloud=WordCloud(font_path="./data/simhei.ttf",background_color="white",max_font_size=80)
word_frequence = {x[0]:x[1] for x in words_count.head(100).values}
wordcloud=wordcloud.fit_words(word_frequence)
plt.imshow(wordcloud)

###  TF-IDF ：提取关键词###

In [ ]:
from gensim import corpora, models, similarities
import gensim
import jieba.analyse
index = 2400
print (df_news['content'][index])
content_S_str = "".join(content_S[index])  
print ("  ".join(jieba.analyse.extract_tags(content_S_str, topK=5, withWeight=False)))

# 将文档中的词进行编号 创建字典
dictionary = corpora.Dictionary(contents_clean)
corpus = [dictionary.doc2bow(sentence) for sentence in contents_clean]

lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20) #类似Kmeans自己指定K值 将字典传入LDA模型 完成LDA主题模型

for topic in lda.print_topics(num_topics=20, num_words=5):
    print (topic[1])

#标签转换为数值
df_train=pd.DataFrame({'contents_clean':contents_clean,'label':df_news['category']})
print(df_train.tail())
df_train.label.unique()
label_mapping = {"汽车": 1, "财经": 2, "科技": 3, "健康": 4, "体育":5, "教育": 6,"文化": 7,"军事": 8,"娱乐": 9,"时尚": 0}
df_train['label'] = df_train['label'].map(label_mapping)
df_train.head()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train['contents_clean'].values, df_train['label'].values, random_state=1)

words = []
for line_index in range(len(x_train)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        words.append(' '.join(x_train[line_index]))
    except:
        print (line_index,word_index) 

In [ ]:
# 将文档内容转换成向量形式 表示每个单词出现的次数
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(analyzer='word', max_features=4000,  lowercase = False)
vec.fit(words)

In [ ]:
# 模型训练
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(words), y_train)

In [ ]:
# 模型测试
test_words = []
for line_index in range(len(x_test)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        test_words.append(' '.join(x_test[line_index]))
    except:
         print (line_index,word_index)
test_words[0]

classifier.score(vec.transform(test_words), y_test)

In [ ]:
# 通过TF-ID转换词模型
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', max_features=4000,  lowercase = False)
vectorizer.fit(words)

#模型训练
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vectorizer.transform(words), y_train)

#模型测试
classifier.score(vectorizer.transform(test_words), y_test)